# 导入数据

In [11]:
"""author Binny"""
# 忽略警告提示
import warnings
warnings.filterwarnings('ignore')
import sys
# 导入处理数据包
import pandas as pd
import numpy as np

train_data = pd.read_csv(r'D:\pythonProject1\ML_for_kaggle\data\titanic\train.csv')
test_data = pd.read_csv(r'D:\pythonProject1\ML_for_kaggle\data\titanic\test.csv')
full = train_data.append(test_data, ignore_index=True)
print('训练集大小：', train_data.shape, '测试集大小：', test_data.shape)
print('合并后的数据集大小：', full.shape)
print('合并后的数据集前5行：\n', full.head())

训练集大小： (891, 12) 测试集大小： (418, 11)
合并后的数据集大小： (1309, 12)
合并后的数据集前5行：
    PassengerId  Survived  Pclass  \
0            1       0.0       3   
1            2       1.0       1   
2            3       1.0       3   
3            4       1.0       1   
4            5       0.0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C1

In [12]:
'''
describe只能查看数据类型的描述统计信息，对于其他类型的数据不显示，比如字符串类型姓名（name），客舱号（Cabin）
这很好理解，因为描述统计指标是计算数值，所以需要该列的数据类型是数据。
Pandas Series.mean()函数返回给定Series对象中基础数据的平均值。
'''
# 获取数据类型列的描述统计信息
full.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,891.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,0.383838,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.486592,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200
